# Análise Exploratória de Dados (EDA) - Dataset de Sepsis

## Objetivo da Análise

* Compreender a estrutura e qualidade dos dados
* Analisar padrões de valores faltantes
* Identificar relações entre variáveis e a ocorrência de sepsis
* Gerar insights para construção de modelos preditivos

## Sobre o Dataset

* **Problema**: Detecção precoce de sepsis em pacientes de UTI
* **Tipo**: Classificação binária (Sepsis vs Não-Sepsis)
* **Características**: Dados de sinais vitais, exames laboratoriais e informações demográficas


## Importação das Bibliotecas

Primeiro, vamos importar todas as bibliotecas necessárias para nossa análise exploratória:

In [ ]:
# Importação das bibliotecas essenciais para análise exploratória
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# Configurações para otimizar as visualizações
warnings.filterwarnings('ignore')
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

# Configuração específica para Google Colab
%matplotlib inline

print("Bibliotecas importadas com sucesso!")

## Carregamento e Estrutura do Dataset

Nesta seção, carregamos o dataset de treino e analisamos suas características básicas para compreender a estrutura dos dados e identificar possíveis problemas de qualidade.

In [ ]:
# Carregamento do dataset de treino
# Para Google Colab: faça upload do arquivo 'dataset_sepsis_train.csv' antes de executar
train_df = pd.read_csv('dataset_sepsis_train.csv')

# Análise da estrutura básica do dataset
print("INFORMAÇÕES GERAIS DO DATASET")
print("=" * 50)
print(f"Dimensões do dataset: {train_df.shape}")
print(f"Número de amostras: {train_df.shape[0]:,}")
print(f"Número de features: {train_df.shape[1]}")
print(f"Tamanho em memória: {train_df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# Verificar tipos de dados
print(f"\nTIPOS DE DADOS")
print("=" * 50)
tipos_dados = train_df.dtypes.value_counts()
for tipo, count in tipos_dados.items():
    print(f"{tipo}: {count} colunas")

# Visualizar as primeiras linhas
print(f"\nPRIMEIRAS 5 LINHAS DO DATASET")
print("=" * 50)
display(train_df.head())

## Separação de Features e Target

Separamos a variável alvo (SepsisLabel) das features para facilitar as análises subsequentes. Esta separação é fundamental para compreender a distribuição das classes e realizar análises bivariadas.

In [ ]:
# Separação entre features (X) e variável target (y)
X_train = train_df.drop('SepsisLabel', axis=1)
y_train = train_df['SepsisLabel']

print(f"Features (X_train): {X_train.shape}")
print(f"Target (y_train): {y_train.shape}")

# Verificar se há duplicatas no dataset
duplicates = train_df.duplicated().sum()
print(f"\nLinhas duplicadas: {duplicates}")

# Análise da distribuição da variável target
print(f"\nDISTRIBUIÇÃO DA VARIÁVEL TARGET")
print("=" * 50)
target_counts = y_train.value_counts().sort_index()
target_pct = y_train.value_counts(normalize=True).sort_index() * 100

for label, count in target_counts.items():
    pct = target_pct[label]
    status = "Não-Sepsis" if label == 0 else "Sepsis"
    print(f"{status} ({label}): {count:,} amostras ({pct:.2f}%)")

# Calcular razão de desbalanceamento
imbalance_ratio = target_counts[0] / target_counts[1]
print(f"\nRazão de desbalanceamento: {imbalance_ratio:.1f}:1")
print(f"Para cada caso de sepsis, existem {imbalance_ratio:.1f} casos de não-sepsis")

## Visualização da Distribuição do Target

A visualização da distribuição da variável target é crucial para compreender o desequilíbrio entre as classes e suas implicações para o desenvolvimento de modelos preditivos.

In [ ]:
# Visualização da distribuição da variável target
plt.figure(figsize=(15, 5))

# Gráfico 1: Contagem absoluta
plt.subplot(1, 3, 1)
bars = plt.bar(['Não-Sepsis', 'Sepsis'], target_counts.values, 
               color=['#3498db', '#e74c3c'], alpha=0.8, edgecolor='black')
plt.title('Distribuição Absoluta das Classes', fontsize=14, pad=20)
plt.xlabel('Classes')
plt.ylabel('Frequência')

# Adicionar valores nas barras
for bar, count in zip(bars, target_counts.values):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(target_counts)*0.01,
            f'{count:,}', ha='center', va='bottom', fontweight='bold')

# Gráfico 2: Distribuição percentual
plt.subplot(1, 3, 2)
bars = plt.bar(['Não-Sepsis', 'Sepsis'], target_pct.values, 
               color=['#3498db', '#e74c3c'], alpha=0.8, edgecolor='black')
plt.title('Distribuição Percentual das Classes', fontsize=14, pad=20)
plt.xlabel('Classes')
plt.ylabel('Percentual (%)')

# Adicionar valores nas barras
for bar, pct in zip(bars, target_pct.values):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(target_pct)*0.01,
            f'{pct:.2f}%', ha='center', va='bottom', fontweight='bold')

# Gráfico 3: Gráfico de pizza
plt.subplot(1, 3, 3)
wedges, texts, autotexts = plt.pie(target_pct.values, 
                                  labels=['Não-Sepsis', 'Sepsis'], 
                                  autopct='%1.2f%%', 
                                  colors=['#3498db', '#e74c3c'],
                                  startangle=90,
                                  explode=(0, 0.1))
plt.title('Proporção das Classes', fontsize=14, pad=20)

# Melhorar formatação dos rótulos
for autotext in autotexts:
    autotext.set_color('white')
    autotext.set_fontweight('bold')

plt.tight_layout()
plt.show()

# Análise das implicações do desbalanceamento
print("IMPLICAÇÕES DO DESBALANCEAMENTO")
print("=" * 50)
if imbalance_ratio > 10:
    print("DATASET ALTAMENTE DESBALANCEADO!")
    print("\nEstratégias recomendadas:")
    print("- Utilizar métricas adequadas (F1-Score, AUC-ROC, Precision-Recall)")
    print("- Aplicar técnicas de balanceamento (SMOTE, undersampling)")
    print("- Configurar class_weight nos algoritmos de ML")
    print("- Considerar threshold optimization")

## Análise de Valores Faltantes

A análise sistemática de valores faltantes é essencial para identificar padrões de missings e definir estratégias de tratamento adequadas. Esta análise nos permite entender se os dados faltam de forma aleatória ou seguem algum padrão específico.

In [ ]:
# Análise quantitativa dos valores faltantes
missing_count = X_train.isnull().sum()
missing_pct = (missing_count / len(X_train)) * 100

# Criar DataFrame para análise organizada
missing_df = pd.DataFrame({
    'Coluna': missing_count.index,
    'Valores_Faltantes': missing_count.values,
    'Porcentagem': missing_pct.values
}).sort_values('Porcentagem', ascending=False)

# Filtrar apenas colunas com valores faltantes
missing_with_nans = missing_df[missing_df['Valores_Faltantes'] > 0]

print("RESUMO DA ANÁLISE DE VALORES FALTANTES")
print("=" * 50)
print(f"Total de colunas: {len(X_train.columns)}")
print(f"Colunas com valores faltantes: {len(missing_with_nans)}")
print(f"Colunas sem valores faltantes: {len(X_train.columns) - len(missing_with_nans)}")

if len(missing_with_nans) > 0:
    print(f"\nTotal de valores faltantes: {missing_count.sum():,}")
    print(f"Porcentagem do dataset: {(missing_count.sum() / (len(X_train) * len(X_train.columns)))*100:.2f}%")
    
    print(f"\nTOP 15 COLUNAS COM MAIOR PORCENTAGEM DE VALORES FALTANTES")
    print("=" * 70)
    display(missing_with_nans.head(15))
else:
    print("Dataset sem valores faltantes detectados!")

## Visualização dos Padrões de Missing Values

A visualização dos padrões de valores faltantes nos ajuda a identificar se existe alguma estrutura nos dados ausentes e a definir estratégias de tratamento mais eficazes.

In [ ]:
# Visualização dos padrões de valores faltantes
if len(missing_with_nans) > 0:
    # Configurar subplots
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # 1. Heatmap dos valores faltantes (top 20 colunas)
    cols_with_nans = missing_with_nans.head(20)['Coluna'].tolist()
    
    if len(cols_with_nans) > 0:
        # Amostra para visualização eficiente
        sample_size = min(1000, len(X_train))
        sample_data = X_train[cols_with_nans].sample(n=sample_size, random_state=42)
        
        sns.heatmap(sample_data.isnull(), 
                   cbar=True, 
                   yticklabels=False,
                   cmap='viridis',
                   ax=axes[0,0])
        axes[0,0].set_title(f'Padrão de Valores Faltantes\n(Top 20 colunas, amostra de {sample_size} linhas)')
        axes[0,0].tick_params(axis='x', rotation=45)
    
    # 2. Gráfico de barras horizontais
    top_missing = missing_with_nans.head(15)
    bars = axes[0,1].barh(range(len(top_missing)), top_missing['Porcentagem'], 
                         color='#2c3e50', alpha=0.7)
    axes[0,1].set_yticks(range(len(top_missing)))
    axes[0,1].set_yticklabels(top_missing['Coluna'])
    axes[0,1].set_xlabel('Porcentagem de Valores Faltantes (%)')
    axes[0,1].set_title('Top 15 Colunas com Maior % de Missing Values')
    axes[0,1].invert_yaxis()
    
    # Adicionar valores nas barras
    for i, bar in enumerate(bars):
        width = bar.get_width()
        axes[0,1].text(width + 1, bar.get_y() + bar.get_height()/2, 
                      f'{width:.1f}%', ha='left', va='center', fontsize=8)
    
    # 3. Histograma da distribuição de missing values
    axes[1,0].hist(missing_with_nans['Porcentagem'], bins=20, alpha=0.7, 
                  color='#34495e', edgecolor='black')
    axes[1,0].set_xlabel('Porcentagem de Missing Values (%)')
    axes[1,0].set_ylabel('Número de Colunas')
    axes[1,0].set_title('Distribuição de Missing Values por Coluna')
    axes[1,0].axvline(missing_with_nans['Porcentagem'].mean(), 
                     color='red', linestyle='--', 
                     label=f'Média: {missing_with_nans["Porcentagem"].mean():.1f}%')
    axes[1,0].legend()
    
    # 4. Categorização dos missing values
    low_missing = missing_with_nans[missing_with_nans['Porcentagem'] < 20]
    medium_missing = missing_with_nans[(missing_with_nans['Porcentagem'] >= 20) & 
                                      (missing_with_nans['Porcentagem'] < 80)]
    high_missing = missing_with_nans[missing_with_nans['Porcentagem'] >= 80]
    
    categories = ['Baixo (<20%)', 'Médio (20-80%)', 'Alto (≥80%)']
    counts = [len(low_missing), len(medium_missing), len(high_missing)]
    colors = ['#27ae60', '#f39c12', '#e74c3c']
    
    wedges, texts, autotexts = axes[1,1].pie(counts, labels=categories, autopct='%1.1f%%', 
                                            colors=colors, startangle=90)
    axes[1,1].set_title('Categorização das Colunas por Nível de Missing Values')
    
    plt.tight_layout()
    plt.show()
    
    # Análise estatística dos missing values
    print("CATEGORIZAÇÃO POR NÍVEL DE MISSING VALUES")
    print("=" * 50)
    print(f"Baixo missing (<20%): {len(low_missing)} colunas")
    print(f"Médio missing (20-80%): {len(medium_missing)} colunas")  
    print(f"Alto missing (≥80%): {len(high_missing)} colunas")
    
    print(f"\nESTRATÉGIAS DE TRATAMENTO RECOMENDADAS")
    print("=" * 50)
    print("Colunas com alto missing (≥80%): Considerar remoção")
    print("Colunas com médio missing (20-80%): Imputação específica por domínio")
    print("Colunas com baixo missing (<20%): Imputação simples (mediana/moda)")
    
else:
    print("Nenhum valor faltante encontrado para visualização!")

## Análise das Variáveis Categóricas

A identificação e análise das variáveis categóricas nos permite compreender as características demográficas e clínicas dos pacientes, bem como sua associação com o desenvolvimento de sepsis.

In [ ]:
# Identificação de variáveis categóricas
print("IDENTIFICAÇÃO DE VARIÁVEIS CATEGÓRICAS")
print("=" * 50)

# Variáveis categóricas explícitas
categorical_cols = X_train.select_dtypes(include=['object', 'category']).columns.tolist()

# Variáveis numéricas que podem ser categóricas (poucos valores únicos)
potential_categorical = []
for col in X_train.columns:
    unique_values = X_train[col].nunique()
    if unique_values <= 10 and X_train[col].dtype in ['int64', 'float64']:
        potential_categorical.append((col, unique_values))

print(f"Variáveis categóricas explícitas: {len(categorical_cols)}")
if categorical_cols:
    print(f"  {categorical_cols}")

print(f"\nVariáveis potencialmente categóricas (≤10 valores únicos): {len(potential_categorical)}")
for col, count in potential_categorical[:10]:
    print(f"  {col}: {count} valores únicos")

# Selecionar variáveis categóricas relevantes para análise
important_categorical = ['Gender', 'Unit1', 'Unit2']
selected_categorical = []

for col in important_categorical:
    if col in X_train.columns:
        selected_categorical.append(col)

# Adicionar outras variáveis categóricas se necessário
for col, count in potential_categorical:
    if col not in selected_categorical and len(selected_categorical) < 5:
        selected_categorical.append(col)

print(f"\nVariáveis selecionadas para análise detalhada: {selected_categorical}")

## Análise Detalhada das Variáveis Categóricas Selecionadas

Vamos examinar estatisticamente cada variável categórica selecionada, incluindo sua distribuição e relação com a variável target.

In [ ]:
# Análise estatística detalhada das variáveis categóricas selecionadas
for col in selected_categorical:
    print(f"\nANÁLISE DA VARIÁVEL: {col}")
    print("=" * 60)
    
    # Distribuição de frequências
    value_counts = X_train[col].value_counts()
    value_pct = X_train[col].value_counts(normalize=True) * 100
    
    print("Distribuição de frequências:")
    for value in value_counts.index:
        count = value_counts[value]
        pct = value_pct[value]
        print(f"  {value}: {count:,} ({pct:.1f}%)")
    
    print(f"\nEstatísticas básicas:")
    print(f"  Valores únicos: {X_train[col].nunique()}")
    print(f"  Valores faltantes: {X_train[col].isnull().sum()}")
    print(f"  Moda: {X_train[col].mode().iloc[0] if len(X_train[col].mode()) > 0 else 'N/A'}")
    
    # Análise da relação com a variável target
    if len(value_counts) <= 10:  # Só para variáveis com poucos valores
        print(f"\nRelação com SepsisLabel:")
        
        # Tabela de contingência
        crosstab = pd.crosstab(X_train[col], y_train, margins=True)
        print("Tabela de contingência:")
        display(crosstab)
        
        # Proporções condicionais
        crosstab_pct = pd.crosstab(X_train[col], y_train, normalize='index') * 100
        print(f"\nTaxa de sepsis por categoria:")
        for category in crosstab_pct.index:
            if category != 'All':
                sepsis_rate = crosstab_pct.loc[category, 1] if 1 in crosstab_pct.columns else 0
                print(f"  {category}: {sepsis_rate:.2f}% de casos com sepsis")
    
    print("-" * 60)

## Visualização das Variáveis Categóricas

As visualizações nos permitem compreender melhor a distribuição das variáveis categóricas e sua associação com a ocorrência de sepsis de forma intuitiva.

In [ ]:
# Visualização das variáveis categóricas
if selected_categorical:
    n_vars = len(selected_categorical)
    fig, axes = plt.subplots(n_vars, 2, figsize=(16, 6*n_vars))
    
    if n_vars == 1:
        axes = axes.reshape(1, -1)
    
    for i, col in enumerate(selected_categorical):
        # Gráfico 1: Distribuição simples
        counts = X_train[col].value_counts()
        
        ax1 = axes[i, 0] if n_vars > 1 else axes[0]
        bars = ax1.bar(range(len(counts)), counts.values, 
                      color='#2c3e50', alpha=0.8, edgecolor='black')
        ax1.set_xticks(range(len(counts)))
        ax1.set_xticklabels(counts.index, rotation=45)
        ax1.set_title(f'Distribuição de {col}')
        ax1.set_xlabel(col)
        ax1.set_ylabel('Frequência')
        
        # Adicionar valores nas barras
        for bar, count in zip(bars, counts.values):
            ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(counts)*0.01,
                    f'{count:,}', ha='center', va='bottom', fontsize=9)
        
        # Gráfico 2: Relação com target
        ax2 = axes[i, 1] if n_vars > 1 else axes[1]
        
        # Criar tabela de contingência para visualização
        temp_df = pd.DataFrame({'categoria': X_train[col], 'target': y_train})
        crosstab = pd.crosstab(temp_df['categoria'], temp_df['target'])
        crosstab_pct = pd.crosstab(temp_df['categoria'], temp_df['target'], normalize='index') * 100
        
        # Gráfico de barras empilhadas
        crosstab.plot(kind='bar', stacked=True, ax=ax2, 
                     color=['#3498db', '#e74c3c'], alpha=0.8)
        ax2.set_title(f'{col} vs SepsisLabel')
        ax2.set_xlabel(col)
        ax2.set_ylabel('Frequência')
        ax2.legend(['Não-Sepsis', 'Sepsis'], title='SepsisLabel')
        ax2.tick_params(axis='x', rotation=45)
        
        # Adicionar percentuais de sepsis
        for j, category in enumerate(crosstab.index):
            if 1 in crosstab_pct.columns:
                sepsis_pct = crosstab_pct.loc[category, 1]
                total_height = crosstab.loc[category].sum()
                ax2.text(j, total_height + max(crosstab.sum(axis=1))*0.02, 
                        f'{sepsis_pct:.1f}%', ha='center', va='bottom', 
                        fontweight='bold', fontsize=8, color='red')
    
    plt.tight_layout()
    plt.show()
    
else:
    print("Nenhuma variável categórica disponível para visualização.")

## Análise de Relações entre Variáveis Categóricas e Numéricas

Esta análise explora como as variáveis categóricas influenciam a distribuição das variáveis numéricas, especialmente no contexto da sepsis. Isso nos ajuda a identificar padrões clínicos importantes.

In [ ]:
# Seleção de variáveis numéricas relevantes para análise
numeric_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()

# Filtrar variáveis numéricas com menos de 50% de valores faltantes
numeric_cols_clean = []
for col in numeric_cols:
    missing_pct = X_train[col].isnull().sum() / len(X_train) * 100
    if missing_pct < 50:
        numeric_cols_clean.append(col)

print("SELEÇÃO DE VARIÁVEIS NUMÉRICAS")
print("=" * 50)
print(f"Total de variáveis numéricas: {len(numeric_cols)}")
print(f"Variáveis com <50% missing: {len(numeric_cols_clean)}")

# Priorizar variáveis de sinais vitais importantes
important_numeric = ['HR', 'SBP', 'DBP', 'Temp', 'Resp', 'O2Sat', 'Age']
selected_numeric = []

for col in important_numeric:
    if col in numeric_cols_clean:
        selected_numeric.append(col)
    if len(selected_numeric) >= 3:
        break

# Completar com outras variáveis se necessário
if len(selected_numeric) < 3:
    for col in numeric_cols_clean:
        if col not in selected_numeric and len(selected_numeric) < 3:
            selected_numeric.append(col)

print(f"Variáveis numéricas selecionadas: {selected_numeric}")
print(f"Variáveis categóricas disponíveis: {selected_categorical}")

## Análise Estatística das Relações Categórica vs Numérica

Vamos examinar estatisticamente como as variáveis categóricas influenciam as distribuições das variáveis numéricas.

In [ ]:
# Análise estatística das relações entre variáveis categóricas e numéricas
for cat_col in selected_categorical[:2]:  # Limitar a 2 categóricas para não sobrecarregar
    for num_col in selected_numeric[:2]:  # Limitar a 2 numéricas por categórica
        
        print(f"\nANÁLISE: {num_col} por {cat_col}")
        print("=" * 60)
        
        # Preparar dados removendo valores faltantes
        temp_df = pd.DataFrame({
            'categorical': X_train[cat_col],
            'numerical': X_train[num_col],
            'target': y_train
        }).dropna()
        
        if len(temp_df) > 0:
            print(f"Amostras válidas para análise: {len(temp_df):,}")
            
            # Estatísticas descritivas por categoria
            stats_by_cat = temp_df.groupby('categorical')['numerical'].agg([
                'count', 'mean', 'std', 'min', 'max', 'median'
            ]).round(2)
            
            print(f"\nEstatísticas descritivas de {num_col} por {cat_col}:")
            display(stats_by_cat)
            
            # Estatísticas por categoria E target (sepsis)
            stats_by_cat_target = temp_df.groupby(['categorical', 'target'])['numerical'].agg([
                'count', 'mean', 'std'
            ]).round(2)
            
            print(f"\nEstatísticas de {num_col} por {cat_col} e SepsisLabel:")
            display(stats_by_cat_target)
            
            # Teste estatístico para diferença de médias (se aplicável)
            categories = temp_df['categorical'].unique()
            if len(categories) == 2:
                from scipy import stats
                
                group1 = temp_df[temp_df['categorical'] == categories[0]]['numerical']
                group2 = temp_df[temp_df['categorical'] == categories[1]]['numerical']
                
                if len(group1) > 1 and len(group2) > 1:
                    # Teste t para amostras independentes
                    t_stat, p_value = stats.ttest_ind(group1, group2, equal_var=False)
                    
                    print(f"\nTeste t para diferença de médias entre categorias:")
                    print(f"Estatística t: {t_stat:.4f}")
                    print(f"P-valor: {p_value:.4f}")
                    
                    alpha = 0.05
                    if p_value < alpha:
                        print(f"Resultado: Diferença estatisticamente significativa (p < {alpha})")
                    else:
                        print(f"Resultado: Diferença não estatisticamente significativa (p >= {alpha})")
            
        else:
            print("Dados insuficientes após remoção de valores faltantes.")
        
        print("-" * 60)

## Visualização das Relações Categórica vs Numérica

Os boxplots nos permitem visualizar como as distribuições das variáveis numéricas variam entre as diferentes categorias, incluindo a presença de sepsis.

In [ ]:
# Visualização das relações entre variáveis categóricas e numéricas
if selected_categorical and selected_numeric:
    # Calcular número de combinações para subplot
    n_combinations = min(4, len(selected_categorical) * len(selected_numeric))
    
    if n_combinations > 0:
        fig, axes = plt.subplots(2, 2, figsize=(16, 12))
        axes = axes.flatten()
        
        combination_count = 0
        
        for cat_col in selected_categorical[:2]:  # Máximo 2 categóricas
            for num_col in selected_numeric[:2]:  # 2 numéricas por categórica
                if combination_count >= n_combinations:
                    break
                
                # Preparar dados removendo valores faltantes
                temp_df = pd.DataFrame({
                    'categorical': X_train[cat_col],
                    'numerical': X_train[num_col],
                    'target': y_train
                }).dropna()
                
                if len(temp_df) > 0:
                    ax = axes[combination_count]
                    
                    # Criar boxplot com separação por target
                    sns.boxplot(data=temp_df, x='categorical', y='numerical', 
                               hue='target', ax=ax, palette=['#3498db', '#e74c3c'])
                    
                    ax.set_title(f'{num_col} por {cat_col} e SepsisLabel', fontsize=12, pad=15)
                    ax.set_xlabel(cat_col)
                    ax.set_ylabel(num_col)
                    ax.tick_params(axis='x', rotation=45)
                    
                    # Configurar legenda
                    handles, labels = ax.get_legend_handles_labels()
                    ax.legend(handles, ['Não-Sepsis', 'Sepsis'], title='SepsisLabel')
                    
                    # Adicionar informação sobre o tamanho da amostra
                    n_points = len(temp_df)
                    ax.text(0.02, 0.98, f'n = {n_points:,}', transform=ax.transAxes, 
                           verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
                
                combination_count += 1
        
        # Remover subplots vazios
        for i in range(combination_count, len(axes)):
            fig.delaxes(axes[i])
        
        plt.tight_layout()
        plt.show()
    
    else:
        print("Combinações insuficientes para visualização.")
        
else:
    print("Variáveis insuficientes para análise de relações.")

## Conclusões da Análise Exploratória

Vamos consolidar os principais insights obtidos durante nossa análise exploratória e suas implicações para o desenvolvimento de modelos preditivos.

In [ ]:
# Consolidação dos principais insights da análise exploratória
print("RESUMO EXECUTIVO DA ANÁLISE EXPLORATÓRIA")
print("=" * 60)

print(f"\n1. ESTRUTURA DO DATASET")
print("-" * 30)
print(f"   • Dimensões: {train_df.shape[0]:,} amostras x {train_df.shape[1]} features")
print(f"   • Tamanho em memória: {train_df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print(f"   • Duplicatas: {train_df.duplicated().sum()} linhas")

print(f"\n2. DISTRIBUIÇÃO DO TARGET")
print("-" * 30)
print(f"   • Não-Sepsis: {target_counts[0]:,} ({target_pct[0]:.2f}%)")
print(f"   • Sepsis: {target_counts[1]:,} ({target_pct[1]:.2f}%)")
print(f"   • Razão de desbalanceamento: {imbalance_ratio:.1f}:1")
print(f"   • Classificação: {'Altamente desbalanceado' if imbalance_ratio > 10 else 'Moderadamente desbalanceado'}")

print(f"\n3. QUALIDADE DOS DADOS")
print("-" * 30)
if len(missing_with_nans) > 0:
    print(f"   • Colunas com missing values: {len(missing_with_nans)}/{len(X_train.columns)}")
    print(f"   • Total de valores faltantes: {missing_count.sum():,}")
    print(f"   • Porcentagem do dataset: {(missing_count.sum() / (len(X_train) * len(X_train.columns)))*100:.2f}%")
    
    # Categorização dos missing values
    low_missing = missing_with_nans[missing_with_nans['Porcentagem'] < 20]
    medium_missing = missing_with_nans[(missing_with_nans['Porcentagem'] >= 20) & 
                                      (missing_with_nans['Porcentagem'] < 80)]
    high_missing = missing_with_nans[missing_with_nans['Porcentagem'] >= 80]
    
    print(f"   • Baixo missing (<20%): {len(low_missing)} colunas")
    print(f"   • Médio missing (20-80%): {len(medium_missing)} colunas")
    print(f"   • Alto missing (≥80%): {len(high_missing)} colunas")
else:
    print(f"   • Dataset sem valores faltantes")

print(f"\n4. VARIÁVEIS CATEGÓRICAS")
print("-" * 30)
print(f"   • Variáveis identificadas: {len(selected_categorical)}")
print(f"   • Variáveis analisadas: {selected_categorical}")

print(f"\n5. VARIÁVEIS NUMÉRICAS")
print("-" * 30) 
print(f"   • Total disponíveis: {len(numeric_cols)}")
print(f"   • Com <50% missing: {len(numeric_cols_clean)}")
print(f"   • Selecionadas para análise: {selected_numeric}")

